In [1]:
import pandas as pd
import numpy as np
import json

# path = "C:\\Users\\gacastil\\Documents\\Clases\\Simulacion\\exercises\\COVID\\"
data_file = "export_dataframe.xlsx"

In [2]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from math import sqrt

ImportError: cannot import name 'ft2font' from partially initialized module 'matplotlib' (most likely due to a circular import) (C:\Users\gacastil\AppData\Roaming\Python\Python38\site-packages\matplotlib\__init__.py)

In [ ]:
xl = pd.ExcelFile(data_file)
covid_df = xl.parse('Sheet1')
covid_df.head()

In [ ]:
covid_df.columns

In [ ]:
covid_df.shape

In [ ]:
covid_df.info()

In [ ]:
covid_df.isnull().sum()

In [ ]:
covid_df.describe()

In [ ]:
covid_df["SEXO"].value_counts()

In [ ]:
labels = covid_df["SEXO"].value_counts().index
values = covid_df["SEXO"].value_counts().values

colors=covid_df["SEXO"]

fig = go.Figure(data=[go.Pie(labels = labels, values=values, textinfo="label+percent", marker=dict(colors=colors))])

fig.show()

In [ ]:
sexo_by_stat = covid_df.groupby(["ENTIDAD_RES"], as_index=False)["ID_REGISTRO"].count()
sexo_by_stat.rename(columns={"ID_REGISTRO": "COUNT"}, inplace=True)
sexo_by_stat.head()

In [ ]:
fig = px.bar(sexo_by_stat, x='ENTIDAD_RES', y='COUNT', text='COUNT')
fig.show()

In [ ]:
sexo_by_mun = covid_df.groupby(["ENTIDAD_RES", "SEXO"], as_index=False)["ID_REGISTRO"].count()
sexo_by_mun.rename(columns={"ID_REGISTRO": "COUNT"}, inplace=True)
sexo_by_mun.head()

In [ ]:
hombres=sexo_by_mun[sexo_by_mun["SEXO"] == "HOMBRE"]
mujeres=sexo_by_mun[sexo_by_mun["SEXO"] == "MUJER"]
fig = go.Figure(data=[
    go.Bar(x=hombres["ENTIDAD_RES"], y=hombres["COUNT"], name="HOMBRE", text=hombres["COUNT"],textposition='auto'),
    go.Bar(x=mujeres["ENTIDAD_RES"], y=mujeres["COUNT"], name="MUJER", text=mujeres["COUNT"],textposition='auto')
])
# Change the bar mode
fig.update_layout(barmode='group')
#fig.update_layout(barmode='stack')
fig.update_layout(
    title='Muertes por estado por sexo en Enero', 
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ), xaxis={'categoryorder':'total ascending'})
# xaxis={'categoryorder':'array', 'categoryarray':['d','a','c','b']}
# xaxis={'categoryorder':'category ascending'}
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [ ]:
acum_dia = covid_df.groupby(["FECHA_INGRESO_DY", "SEXO"], as_index=False)["ID_REGISTRO"].count()
acum_dia.rename(columns={"ID_REGISTRO": "COUNT"}, inplace=True)
acum_dia

In [ ]:
fig = px.line(acum_dia, x="FECHA_INGRESO_DY", y="COUNT", color="SEXO")
fig.show()

In [ ]:
type(sexo_by_mun)

In [ ]:
covid_df[covid_df["EDAD"] > 60]

In [ ]:
sub_covid_df = covid_df[["ID_REGISTRO", "SEXO", "ENTIDAD_RES", "MUNICIPIO_RES", "EDAD", "FECHA_DEF"]]
sub_covid_df.head()

In [ ]:
# Top 10 - Estados con mayores muertes divididos por hombre y mujer 
contagio_hom = sub_covid_df[sub_covid_df["SEXO"] == "HOMBRE"].groupby("ENTIDAD_RES", as_index=False)["ID_REGISTRO"].count()
contagio_hom.rename(columns={"ID_REGISTRO": "HOMBRE"}, inplace=True)
contagio_muj = sub_covid_df[sub_covid_df["SEXO"] == "MUJER"].groupby("ENTIDAD_RES", as_index=False)["ID_REGISTRO"].count()
contagio_muj.rename(columns={"ID_REGISTRO": "MUJER"}, inplace=True)
#tot = pd.concat([th["ID_REGISTRO"], tm["ID_REGISTRO"]], axis=1,index="ENTIDAD_RES", keys=["HOMBRE","MUJER"])
tot_contagio = contagio_hom.merge(contagio_muj, how='inner', on='ENTIDAD_RES')
tot_contagio["TOTAL"] = tot_contagio["HOMBRE"] + tot_contagio["MUJER"]
tot_contagio = tot_contagio.sort_values(by="TOTAL", ascending=False)[:10]
tot_contagio

In [ ]:
labels = tot_contagio["ENTIDAD_RES"]

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=tot_contagio["HOMBRE"], name="Hombres"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=tot_contagio["MUJER"], name="Mujeres"), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Top 10 contagios por estado",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Hombres', x=0.17, y=0.5, font_size=20, showarrow=False),
                 dict(text='Mujeres', x=0.83, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
tot_contagio.iloc[0:4]["ENTIDAD_RES"]

In [ ]:
# Por estado hombres vs mujeres
labels = ["HOMBRES", "MUJERES"]
# Create subplots, using 'domain' type for pie charts
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs, subplot_titles=tot_contagio.iloc[0:4]["ENTIDAD_RES"].values)

# Define pie charts
fig.add_trace(go.Pie(labels=labels, values=tot_contagio.iloc[0]["HOMBRE":"MUJER"], name=tot_contagio.iloc[0]["ENTIDAD_RES"]), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=tot_contagio.iloc[1]["HOMBRE":"MUJER"], name=tot_contagio.iloc[1]["ENTIDAD_RES"]), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=tot_contagio.iloc[2]["HOMBRE":"MUJER"], name=tot_contagio.iloc[2]["ENTIDAD_RES"]), 2, 1)
fig.add_trace(go.Pie(labels=labels, values=tot_contagio.iloc[3]["HOMBRE":"MUJER"], name=tot_contagio.iloc[3]["ENTIDAD_RES"]), 2, 2)


# Tune layout and hover info
fig.update_traces(hoverinfo='label+percent+name')
fig.update(layout_title_text='Top 4 states by sex')

fig = go.Figure(fig)
fig.show()

In [ ]:
sub_count = sub_covid_df.groupby(["ENTIDAD_RES", "MUNICIPIO_RES", "SEXO"], as_index=False)["ID_REGISTRO"].count()
sub_count.rename(columns={"ID_REGISTRO": "COUNT"}, inplace=True)
sub_count["CTRY"] = "MEX"
sub_count["AGE"] = sub_covid_df.groupby(["ENTIDAD_RES", "MUNICIPIO_RES", "SEXO"])["EDAD"].mean().values
sub_count.head()

In [ ]:
sub_count[(sub_count["COUNT"] > 10) | (sub_count["ENTIDAD_RES"].isin(["DF","NL", "JC"]))].tail(10)
# sub_count[sub_count["COUNT"] > 5].groupby("ENTIDAD_RES")["COUNT"].sum()

In [ ]:
sub_count.to_json(orient="records")

In [ ]:
sub_count[(sub_count["ENTIDAD_RES"].isin(["DF","NL", "JC"]))].tail(10)

In [ ]:
states = ["DF","NL","JC"]

labels_ct = sub_count[sub_count["ENTIDAD_RES"].isin(states)]["MUNICIPIO_RES"]
parents_ct = sub_count[sub_count["ENTIDAD_RES"].isin(states)]["ENTIDAD_RES"]
values_ct = sub_count[sub_count["ENTIDAD_RES"].isin(states)]["COUNT"]

labels_st = tot_contagio[tot_contagio["ENTIDAD_RES"].isin(states)]["ENTIDAD_RES"]
parents_st = pd.Series(['MEX']).repeat(labels_st.size)
values_st = tot_contagio[tot_contagio["ENTIDAD_RES"].isin(states)]["TOTAL"]

labels_rt = pd.Series(['MEXICO'])
parents_rt = pd.Series([''])
values_rt = pd.Series(['100'])

labelsx = labels_rt.append(labels_st, ignore_index=True).append(labels_ct, ignore_index=True)
parentsx = parents_rt.append(parents_st, ignore_index=True).append(parents_ct, ignore_index=True)
valuesx = values_rt.append(values_st, ignore_index=True).append(values_ct, ignore_index=True)

data = dict(regions = labelsx, hierarchy = parentsx, number = valuesx)

# fig =px.sunburst(data, names='regions', parents='hierarchy', values='number')
# fig =go.Figure()
# fig.add_trace(go.Sunburst(labels=labelsx, parents=parentsx, values=valuesx, ))
# fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig = px.sunburst(sub_count[sub_count["ENTIDAD_RES"].isin(states)], 
                  path=['CTRY', 'ENTIDAD_RES', 'MUNICIPIO_RES'], 
                  values='COUNT', color='AGE', color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(sub_count['AGE']))
fig.show()

#str(labelsx.size) + "-" + str(parentsx.size) + "-" + str(valuesx.size)

In [ ]:
sub_covid_df.head()

In [ ]:
# Rango de edades de infectados
fig = px.box(sub_covid_df, x="ENTIDAD_RES", y="EDAD")
fig.show()

In [ ]:
# Puntos muestrales
fig = px.strip(sub_covid_df, x="ENTIDAD_RES", y="EDAD")
fig.show()

In [ ]:
# box plot comparing ages of men vs women by state

fig = go.Figure()

fig.add_trace(go.Box(
    x=sub_covid_df[sub_covid_df["SEXO"] == "HOMBRE"]["EDAD"],
    y=sub_covid_df["ENTIDAD_RES"],
    name='Hombres',
    marker_color='#3D9970'
))
fig.add_trace(go.Box(
    x=sub_covid_df[sub_covid_df["SEXO"] == "MUJER"]["EDAD"],
    y=sub_covid_df["ENTIDAD_RES"],
    name='Mujeres',
    marker_color='#FF4136'
))

fig.update_layout(
    yaxis_title='edad',
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.update_traces(orientation='h')
fig.show()

In [ ]:
# edad en la población en general
fig = px.violin(sub_covid_df, y="EDAD", color="SEXO",
                violinmode='overlay', # draw violins on top of each other
                # default violinmode is 'group' as in example above
                hover_data=sub_covid_df.columns)
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Violin(x=sub_covid_df['ENTIDAD_RES'][ sub_covid_df['SEXO'] == 'HOMBRE' ],
                        y=sub_covid_df['EDAD'][ sub_covid_df['SEXO'] == 'HOMBRE' ],
                        legendgroup='Yes', scalegroup='Yes', name='Yes',
                        side='negative',
                        line_color='blue')
             )
fig.add_trace(go.Violin(x=sub_covid_df['ENTIDAD_RES'][ sub_covid_df['SEXO'] == 'MUJER' ],
                        y=sub_covid_df['EDAD'][ sub_covid_df['SEXO'] == 'MUJER' ],
                        legendgroup='No', scalegroup='No', name='No',
                        side='positive',
                        line_color='orange')
             )
fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0, violinmode='overlay')
fig.show()